<a href="https://colab.research.google.com/github/HelgaPotapenko/DataS_Homeworks/blob/main/W2V_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [114]:
from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models.callbacks import PerplexityMetric

from nltk.corpus import stopwords
from nltk.tokenize import WordPunctTokenizer
from nltk.stem import SnowballStemmer
from pymorphy2 import MorphAnalyzer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np
import sqlite3
import re
from datetime import datetime

from tqdm import tqdm
tqdm.pandas()

from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()

In [3]:
#pip install pymorphy2

In [4]:
#import nltk
#nltk.download('stopwords')

In [5]:
#pip install pandas jupyter pandarallel requests tqdm#

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [78]:
global_params = {
    'random_rows_cnt':1000
    , 'http_success':200
    , 'rows_return':10
    , 'w2v_model_file':'/content/drive/My Drive/word2vec.model'
    , 'd2v_model_file':'/content/drive/My Drive/doc2vec.model'
    , 'db_name_data':'/content/drive/My Drive/Test_data.db'
    , 'db_name_processing':'/content/drive/My Drive/Test_data.db'
    , 'full_text_table':'vacancies_text'

    }

In [8]:
# селектит и возвращает предобработанные тексты вакансий из базы
#p_limit - количество возвращаемых записей. Если 0 - вернуть все записи
def get_preproc_vacancies_data(p_limit, p_table_name):
    c_db_name_processing = global_params.get('db_name_processing')

    db_conn = sqlite3.connect(c_db_name_processing)
    sql_rq = f"SELECT id, preproc  FROM {p_table_name}"
    if p_limit>0:
      sql_rq += f" order by random() limit {p_limit}"
    df_data = pd.read_sql_query(sql_rq, con=db_conn)
    db_conn.close()

    df_data['preproc']=df_data['preproc'].apply(lambda x: x.split(';'))

    #####убрать после отладки
    print ("считал", len(df_data.index))
    startTime = datetime.now()
    #####

    return (df_data[['id','preproc']])


In [9]:
#стемминг, лемматизация, удаление стоп-слов
def normalize(p_strs, p_tokenizer, p_lemmatizer, p_steamer, p_stop_words):
    preproc_str=[]
    for w in p_tokenizer.tokenize(p_strs):
        w = p_steamer.stem(p_lemmatizer.normal_forms(w)[0])
        if (re.match('^\w+', w)) and (w not in p_stop_words):
            preproc_str.append(w)
    res = ';'.join(preproc_str)
    return res

In [10]:
#предобработка данных
#Получает из базы FullText вакансии
#результат записывает в таблицу p_result_table_name
def preproc_vacancies_data(p_result_table_name):

  c_db_name_processing = global_params.get('db_name_processing')
  c_db_name_data = global_params['db_name_data'] 
  c_full_text_table = global_params.get('full_text_table')
  c_rows_pack_num = 10000 #будем обрабатывать порциями по c_rows_pack_num строк

  steamer = SnowballStemmer(language="russian") 
  lemmatizer = MorphAnalyzer()
  tokenizer = WordPunctTokenizer()
  stop_words = stopwords.words("russian")

  #очищаю целевую таблицу
  db_conn = sqlite3.connect(c_db_name_processing)
  db_cursor = db_conn.cursor()
  sql_rq = f"""create table if not exists {p_result_table_name}(id integer, preproc text)"""
  db_cursor.execute(sql_rq)
  db_conn.commit()
  sql_rq = f'delete from {p_result_table_name}'
  db_cursor.execute(sql_rq)
  db_conn.commit()
  db_conn.close() 

  #####убрать после отладки
  startTime = datetime.now()
  #####

  v_offset = 0
  while True:
      #считываю пачку FullText вакансий
      db_conn = sqlite3.connect(c_db_name_data)
      sql_rq = f"""SELECT id, FullText  FROM {c_full_text_table} limit {c_rows_pack_num} offset {v_offset}"""
      df_data = pd.read_sql_query(sql_rq, con=db_conn)
      db_conn.close()

      v_offset += len(df_data.index)
      if len(df_data.index)==0:
        break

      #предобработка
      tqdm.pandas()
      df_data['preproc'] = df_data['FullText'].progress_apply(lambda x: normalize(x, tokenizer, lemmatizer, steamer, stop_words))

      #запись предобработанных данных во временную таблицу
      v_tmp_table_name = p_result_table_name+'_tmp'
      db_conn = sqlite3.connect(c_db_name_processing)
      df_data[['id','preproc']].to_sql(v_tmp_table_name
                                      , con=db_conn
                                      , if_exists='replace'
                                      , index = False
                                      , dtype = {"id": "INTEGER"})

      #перенос из временной таблицы в целевую
      db_cursor = db_conn.cursor()

      v_fields = 'id, preproc'
      sql_rq = f'insert into {p_result_table_name} ({v_fields}) select {v_fields} from  {v_tmp_table_name}'
      db_cursor.execute(sql_rq)
      db_conn.commit()
      
      db_conn.close() 

  #####убрать после отладки
  print ("Предобработано за ", datetime.now()-startTime)
  #####
  return v_offset

In [22]:
#get_preproc_vacancies_data(10, 'vacancies_preproc')

In [12]:
#обучение W2V
def w2v_trian(p_model_file, p_epochs):
    с_epochs = 10
    c_vector_size = 100
    df_train_data=get_preproc_vacancies_data(0, 'vacancies_preproc')
    #min_count — игнорировать все слова с частотой встречаемости меньше, чем это значени
    w2v_model = Word2Vec(vector_size=100, min_count=1)
    
    #обучение
    w2v_model.build_vocab(df_train_data['preproc'])
    w2v_model.train(df_train_data['preproc'], total_examples=w2v_model.corpus_count, epochs=p_epochs, report_delay=1)
    #сохранение модели
    w2v_model.save(p_model_file)

    return w2v_model

In [23]:
#возвращает список солв из p_list_of_words, закодированных w2v-моделью p_model
def get_w2v_embeddings(p_model, p_list_of_words):
  res = []
  for word in p_list_of_words:
    res.append(p_model.wv[word])

  return(res)

In [27]:
#Идея: преобразовать каждое слово в описании вакансии в w2v-представление, аналогично преобразовать резюме.
# Вычислить средний вектор для каждого описания вакансии. Сравнить с данными резюме по косиносному расстоянию
# p_text - данные для поиска (из резюме)
# p_recalc_flag - признак, надо ли пересчитывать предобработку данных по вакансиям и заново обучать W2V-модель
    # 0 - ничего не пересчитывать
    # 1 - пересчитать только w2v
    # 2 - предобработать данные и пересчитать w2v
def word2vec_mean_get_top(p_text, p_recalc_flag=0):
    c_random_rows_cnt = global_params.get('random_rows_cnt')
    c_rows_return = global_params.get('rows_return')
    c_model_file = global_params.get('w2v_model_file')
    c_db_name_data = global_params.get('db_name_data')
    c_db_name_processing = global_params.get('db_name_processing')
    c_epochs = 10
          
    steamer = SnowballStemmer(language="russian") 
    lemmatizer = MorphAnalyzer()
    tokenizer = WordPunctTokenizer()
    stop_words = stopwords.words("russian")

    #обучение W2V, если выставлен флаг пересчета
    #либо подгрузка из БД, если флаг не стоит
    if p_recalc_flag == 2:
      preproc_vacancies_data('vacancies_preproc')
      w2v_model = w2v_trian(c_model_file, c_epochs)
    elif  p_recalc_flag == 1:
      w2v_model = w2v_trian(c_model_file, c_epochs)
    else:
      w2v_model = Word2Vec.load(c_model_file)

    #получение данных
    df_data=get_preproc_vacancies_data(c_random_rows_cnt, 'vacancies_preproc')
    print (len(df_data.index))

    #предобработка резюме
    searched_preproc = normalize(p_text, tokenizer, lemmatizer, steamer, stop_words).split(';')

    #дополняю модель W2V новыми словами
    w2v_model.build_vocab([searched_preproc], update=True)
    w2v_model.train([searched_preproc], total_examples=1, epochs=1)

    #считаю средний вектор по документу
    df_data['mean_W2V'] = df_data['preproc'].apply(lambda x: np.mean(get_w2v_embeddings(w2v_model, x), axis=0))
    searched_vec = get_w2v_embeddings(w2v_model, searched_preproc)  

    #определяю схожесть вакансий с резюме
    df_data['cos_dist'] = df_data['mean_W2V'].apply(lambda x: cosine_similarity(searched_vec, x.reshape(1,-1))[0][0])

    return (df_data.sort_values(by='cos_dist', ascending = False)[['id','preproc','cos_dist']].head(c_rows_return))

In [122]:
#обучение D2V
def d2v_trian(p_model_file, p_epochs, p_vector_size):
    df_train_data=get_preproc_vacancies_data(0, 'vacancies_preproc')   
    v_train_data = [TaggedDocument(doc, [i]) for i, doc in enumerate(df_train_data['preproc'])]
    d2v_model = Doc2Vec(v_train_data, vector_size=p_vector_size, min_count=1)
    #обучение
    d2v_model.build_vocab(v_train_data)
    d2v_model.train(v_train_data, total_examples=d2v_model.corpus_count, epochs=p_epochs, report_delay=1)
    #сохранение модели
    d2v_model.save(p_model_file)

    return d2v_model

In [119]:
#Идея: с помощью doc2vec вычислить вектор для каждого описания вакансии. Сравнить с данными резюме по косиносному расстоянию
# p_text - данные для поиска (из резюме)
# p_recalc_flag - признак, надо ли пересчитывать предобработку данных по вакансиям и заново обучать D2V-модель
    # 0 - ничего не пересчитывать
    # 1 - пересчитать только d2v
    # 2 - предобработать данные и пересчитать d2v
def doc2vec_get_top(p_text, p_recalc_flag=0):
    c_random_rows_cnt = global_params.get('random_rows_cnt')
    c_rows_return = global_params.get('rows_return')
    c_model_file = global_params.get('d2v_model_file')
    c_db_name_data = global_params.get('db_name_data')
    c_db_name_processing = global_params.get('db_name_processing')
    c_epochs = 2
    c_vector_size = 100
          
    steamer = SnowballStemmer(language="russian") 
    lemmatizer = MorphAnalyzer()
    tokenizer = WordPunctTokenizer()
    stop_words = stopwords.words("russian")

    #обучение D2V, если выставлен флаг пересчета
    #либо подгрузка из БД, если флаг не стоит
    if p_recalc_flag == 2:
      preproc_vacancies_data('vacancies_preproc')
      d2v_model = d2v_trian(c_model_file, c_epochs, c_vector_size )
    elif  p_recalc_flag == 1:
      d2v_model = d2v_trian(c_model_file, c_epochs, c_vector_size)
    else:
      d2v_model = Doc2Vec.load(c_model_file)

    #получение данных
    df_data=get_preproc_vacancies_data(c_random_rows_cnt, 'vacancies_preproc')
    print (len(df_data.index))

    #вычисление вектора d2v для каждой полученной записи
    df_data['D2V'] = df_data['preproc'].apply(lambda x: d2v_model.infer_vector(x).reshape(1,-1))

    #предобработка резюме
    searched_preproc = normalize(p_text, tokenizer, lemmatizer, steamer, stop_words).split(';')
    searched_vec = d2v_model.infer_vector(searched_preproc).reshape(1,-1)

    #определяю схожесть вакансий с резюме
    df_data['cos_dist'] = df_data['D2V'].apply(lambda x: cosine_similarity(searched_vec, x)[0][0])

    return (df_data.sort_values(by='cos_dist', ascending = False)[['id','preproc','cos_dist']].head(c_rows_return))

In [123]:
pd.set_option('max_colwidth', 0)
text = 'дерматолог высшей категори'
word2vec_mean_get_top(text, p_recalc_flag=0)

считал 1000
1000


,id,preproc,cos_dist
366,77266636,"[врач, педиатр, участков, земск, доктор, оп, полн, ден, полн, занят, квалифицирова, оказан, медицинск, помощ, рамк, амбулаторн, поликлиническ, практик, согласн, стандарт, медицинск, обслуживан, комплексн, подход, лечен, пациент, направлен, друг, специалист, поликлиник, веден, первичн, медицинск, документац, организац, контрол, работ, средн, медицинск, персона, педиатрическ, отделен, поликлиник, оказан, квалифицирова, медицинск, помощ, использова, современ, метод, профилактик, диагностик, лечен, реабилитац, разреш, применен, медицинск, практик, определен, тактик, веден, больн, соответств, установлен, прав, стандарт, амбулаторн, пациент, поликлиник, дом, оказан, медицинск, помощ, пациент, остр, состоян, оформлен, листок, времен, нетрудоспособн, оформлен, рецепт, пациент, наход, дло, требован, дипл, высок, медицинск, образован, сертификат, quot, педиатр, quot, владен, навык, врач, педиатр, участков, знан, ...]",0.472136
890,77714822,"[акушерк, акушер, акушерск, отделен, оп, полн, ден, полн, занят, обязан, провод, лечебн, профилактическ, санитарн, просветительск, работ, уход, пациент, соответств, профил, отделен, руководств, врач, осуществля, лечебн, диагностическ, помощ, беремен, рожениц, назанчен, врач, ил, совместн, отделен, требован, средн, профессиональн, образован, специальн, quot, лечебн, дел, quot, quot, сестринск, дел, quot, quot, акушерск, дел, quot, сертификат, специалист, специальн, quot, акушерск, дел, quot, услов, структур, 12, вход, бол, 70, подразделен, поликлиник, амбулатор, фап, небольш, чист, уютн, зелен, город, приглаша, современ, прекрасн, развит, инфраструктур, ребенок, взросл, школ, детск, сад, школ, искусств, городск, парк, кинотеатр, биатлон, центр, каток, современ, бассейн, центр, детск, творчеств, дворец, культур, горяч, источник, ...]",0.466625
568,78346787,"[врач, хирург, 1, год, 3, год, полн, ден, полн, занят, обязан, хирург, занима, диагностик, заболеван, обязан, вход, осмотр, пациент, назначен, изучен, анализ, исследован, биопс, уз, рентгенограф, кт, мрт, друг, консультирован, наблюден, больн, назначен, терап, подготовк, человек, операц, реабилитац, посл, требован, врач, хирург, обяза, имет, высок, образован, специалитет, специальн, лечебн, дел, педиатр, подготовк, интернатур, ординатур, специальн, хирург, требован, такж, обязыва, стандартн, проход, врач, хирург, повышен, квалификац, редк, 5, год, течен, ве, трудов, деятельн, кулунд, врач, хирург, обяза, имет, высок, образован, специалитет, специальн, лечебн, дел, педиатр, подготовк, интернатур, ординатур]",0.457521
372,78141091,"[врач, ультразвуков, диагностик, устьянск, центральн, район, больниц, оп, полн, ден, полн, занят, требова, рач, ультразвуков, диагностик, гбуз, архангельск, област, quot, устьянск, центральн, район, больниц, quot, обязан, соответств, должностн, инструкц, требован, 1, высок, образован, специалитет, специальн, quot, лечебн, дел, quot, quot, педиатр, quot, quot, медицинск, биофизик, quot, ил, quot, медицинск, кибернетик, quot, лиц, заверш, образован, 2018, год, прошедш, подготовк, ординатур, специальн, quot, ультразвуков, диагностик, quot, ил, имет, высок, образован, специалитет, специальн, quot, лечебн, дел, quot, quot, педиатр, quot, quot, медицинск, биофизик, quot, ил, quot, медицинск, кибернетик, quot, прошедш, подготовк, интернатур, ил, ординатур, специальн, quot, авиацион, космическ, медицин, quot, quot, акушерств, гинеколог, ...]",0.451961
76,73757697,"[врач, акушер, гинеколог, оп, полн, ден, полн, занят, обязан, оказан, медицинск, помощ, населен, профил, quot, акушерств, гинеколог, quot, стационарн, услов, проведен, медицинск, обследован, пациент, период, беремен, род, послеродов, период, посл, прерыван, беремен, гинекологическ, заболеван, доброкачествен, диффузн, изменен, молочн, желез, стационарн, услов, назначен, проведен, лечен, пациент, период, беремен, род, послеродов, период, посл, прерыван, беремен, гинекологическ, заболеван, доброкачествен, диффузн, изменен, молочн, желез, ко

In [124]:
text = 'Врач-дерматолог высшей категори'
doc2vec_get_top(text, p_recalc_flag=1)

считал 90000


считал 1000
1000


,id,preproc,cos_dist
395,78008776,"[медицинск, представител, 1, год, 3, год, полн, ден, полн, занят, обязан, продвижен, продукц, компан, поиск, привлечен, клиент, проведен, виз, врач, лпу, работник, аптек, выполнен, план, продаж, веден, отчетн, документац, выполнен, поручен, руководств, требован, высок, медицинск, образован, специальн, врач, собеседован, имет, коп, дипл, отличн, знан, русск, узбекск, язык, знан, английск, язык, приветствова, коммуникабельн, честност, исполнительн, ответствен, открыт, решительн, организован, умен, работа, команд, навык, работ, компьютер, планшет, word, excel, power, point, оп, работ, дат, должност, приветствова, услов, полн, рабоч, ден, 09, 00, 18, 00, перер, обед, ве, рабоч, процесс, дублирова, срм, систем, навык, продаж, работ, текущ, баз, клиент, навык, презентац, навык, межличностн, ...]",0.450767
719,78046180,"[врач, офтальмолог, 1, год, 3, год, полн, ден, полн, занят, команд, профессиона, 3z, офтальмолог, явля, крупн, сет, офтальмологическ, центр, росс, бол, 18, год, оказыва, высококвалифицирова, помощ, пациент, наш, мисс, искореня, привычк, человек, плох, зрен, предлага, решен, сервис, высок, уровен, наход, поиск, врач, офтальмолог, предстоя, занима, проведен, полн, диагностик, состоян, глаз, пациент, постановк, диагноз, основан, дан, обследован, назначен, необходим, лечен, направлен, хирургическ, лечен, показан, оформлен, медицинск, документац, соответств, действова, стандарт, амбулаторн, карт, работ, медицинск, информацион, систем, ожида, высок, медицинск, образован, действова, медицинск, сертификат, оп, работ, врач, офтальмолог, гарантирова, работ, федеральн, компан, оформлен, согласн, тк, рф, официальн, заработн, плат, выплат, дважд, месяц, ...]",0.419893
768,78050628,"[специалист, пункт, выдач, заказ, стрелецк, 70, 4, оп, полн, ден, полн, занят, quot, всеинструмент, ру, эт, дом, дач, стройк, ремонт, quot, сегодн, наш, плеч, 16, год, развит, рост, результат, благодар, наш, сотрудник, вход, топ, 10, крупн, интернет, магазин, росс, возглавля, рейтинг, e, commerce, игрок, 1, категор, дом, дач, строительств, ремонт, рейтинг, data, insight, наш, команд, эт, бол, 7000, счастлив, сотрудник, такж, 8, мест, росс, индекс, лояльност, сотрудник, вход, топ, 20, рейтинг, хорош, работодател, росс, крупн, компан, настоя, момент, наш, команд, поиск, специалист, пункт, выдач, заказ, интересова, инструмент, быт, преимуществ, оп, работ, позиц, продавец, консультант, продавец, кассир, менеджер, продаж, менеджер, работ, ...]",0.393096
760,77745925,"[врач, эпидемиолог, оп, полн, ден, полн, занят, обязан, осуществля, работ, соответств, план, мероприят, профилактик, инфекцион, заболеван, организова, контролирова, выполнен, профилактическ, противоэпидемическ, мероприят, контролирова, своевремен, выявлен, учет, инфекцион, заболеван, проведен, необходим, лабораторн, обследован, лечен, больн, первичн, противоэпидемическ, мероприят, провод, текущ, ретроспективн, анализ, инфекцион, заболеваем, определя, причин, следствен, связ, совместн, территориальн, центр, госсанэпиднадзор, требован, высок, медицинск, образован, прошедш, послевузовск, подготовк, ил, специализац, специальн, quot, эпидемиолог, quot, услов, структур, 12, вход, бол, 70, подразделен, поликлиник, амбулатор, фап, наш, команд, высококвалифицирова, специалист, приня, наш, дружн, коллект, помоч, адаптац, нов, мест, разв, систем, наставничеств, опытн, наставник, подел, сво, оп, помоч, усовершенствова, сво, навык, участ, программ, ...]",0.391962
698,78391299,"[мастер, контрол, техническ, состоян, автомобил, 1, год, 3, год, полн, ден, полн, занят, обязан, проверк, техническ, состоян, автотранспортн, средств, выезд, лин, возвращен, лин, ежедневн, осуществлен, руководств, подвижн, соста, автоколон, контрол, маршрутизац, рейс, цел, наибол, эффективн, расход, топлив, ежедневн, разбор, проверк, правильн, расход, топлив, водительск, соста, контрол, исполнен, работник, требован, охра, труд, пожарн, безопасн, осуществлен, контрол, качеств, своевремен, выполнен, раб

In [95]:
##https://medium.com/wisio/a-gentle-introduction-to-doc2vec-db3e8c0cce5e
##сделать doc2vec c тегами